In [1]:
%run ./_library/packages.py

# Load the necessary data files
with open('../../model/documents_dict.json', 'r', encoding='utf-8') as f:
    documents = json.load(f)

with open('../../model/encoded_segments.json', 'r', encoding='utf-8') as f:
    encoded_segments = json.load(f)

with open('../../model/segment_encodings.json', 'r', encoding='utf-8') as f:
    segment_encodings = json.load(f)  

with open('../../model/sat_segments_dict.json', 'r', encoding='utf-8') as f:
    sat_segments_dict = json.load(f)

<frozen importlib._bootstrap>:241: RuntimeWarning: compiletime version 3.9 of module 'angular_distance' does not match runtime version 3.1
<frozen importlib._bootstrap>:241: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 880 from C header, got 888 from PyObject


In [3]:
# Creating list of SAT topics.
#topics = list(sat_segments_dict.keys())
topics = ['referen']

In [ ]:
total_TP = 0
total_FN = 0
total_FP = 0
total_TN = 0

for sat_topic in topics:    
    # Print selected SAT topic
    print('Selected SAT:', sat_topic)

    # Step 1: Get the encoding vectors for the SAT
    sat_segment_ids = sat_segments_dict[sat_topic]
    sat_segment_indices = [encoded_segments.index(segment_id) for segment_id in sat_segment_ids]
    sat_encodings = [segment_encodings[index] for index in sat_segment_indices]

    print('Number of segments in SAT:', len(sat_segment_ids))
    print()

    # Step 2: Build a similarity matrix with SAT segments in rows and corpus segments in columns
    print('Building matrix…')
    sim_matrix = cdist(sat_encodings, segment_encodings, ad.angular_distance)  # Cosine similarity
    print('Similarity matrix dimensions:', sim_matrix.shape)
    print()

    # Step 3: Set a similarity threshold for classification
    threshold = 0.8

    # Step 4: Initialize counters for True Positives (TP), False Negatives (FN), False Positives (FP), and True Negatives (TN)
    TP = 0
    FN = 0
    FP = 0
    TN = 0

    # Step 5: For each segment, classify based on similarity threshold
    for idx, sentence_id in enumerate(encoded_segments):
        # Get the similarity values for each SAT segment to the current sentence
        similarities = sim_matrix[:, idx]

        # Segments predicted to be similar (above threshold)
        predicted_similar = similarities >= threshold

        # Segments that are actually part of the SAT topic (actual similar)
        actual_similar = sentence_id in sat_segments_dict[sat_topic]

        # True Positives (TP): Segments that were correctly predicted as similar to the SAT topic
        if predicted_similar.any() and actual_similar:
            TP += 1

        # False Negatives (FN): Segments that are part of the SAT topic but predicted below the threshold
        if not predicted_similar.any() and actual_similar:
            FN += 1

        # False Positives (FP): Segments that were incorrectly predicted as similar to the SAT topic
        if predicted_similar.any() and not actual_similar:
            FP += 1

        # True Negatives (TN): Segments that were correctly predicted as not similar to the SAT topic
        if not predicted_similar.any() and not actual_similar:
            TN += 1

    # Output the results
    print(f"True Positives (TP): {TP}")
    print(f"False Negatives (FN): {FN}")
    print(f"False Positives (FP): {FP}")
    print(f"True Negatives (TN): {TN}")

Selected SAT: referen
Number of segments in SAT: 348

Building matrix…
